In [1]:
import psycopg2
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
stopWords = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
import re
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, SVD
import seaborn as sns
from surprise.model_selection import cross_validate
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


# Connect to the database
conn = psycopg2.connect(
    host="localhost",
    port=5432,
    database="newsApp",
    user="user1",
    password="user123"
)

cur = conn.cursor()

cur.execute("SELECT * FROM articles")

In [2]:

data = cur.fetchall()

articles = pd.DataFrame(data, columns= ['article_id', 'source_id', 'source_name', 'author', 'title', 'description', 'url', 'url_to_image', 'published_at', 'content', 'category', 'category_specified'])

cur.execute("SELECT * FROM disliked_posts WHERE user_id = 4")

dislikedPostsUser = cur.fetchall()

dislikedPosts = pd.DataFrame(dislikedPostsUser, columns= ['dislike_id', 'user_id', 'article_id', 'disliked'])

cur.execute("SELECT * FROM liked_posts WHERE user_id = 4")

liked = cur.fetchall()

likedPosts = pd.DataFrame(liked, columns = ['liked_posts_id', 'user_id', 'article_id', 'liked'])

cur.execute("SELECT * FROM save_for_later WHERE user_id = 4")

save_for_laterposts = cur.fetchall()

save_for_laterPosts = pd.DataFrame(save_for_laterposts, columns = ['save_for_later_id', 'user_id', 'article_id', 'save_for_later'])

cur.execute("SELECT * from user_subject_preferences where user_id = 4")

preferences = cur.fetchall()

preferenceProfile = pd.DataFrame(preferences, columns = ['user_id','business', 'politics', 'right_wing', 'left_wing', 'center', 'sports', 'nfl','nba','nhl','mma', 'entertainment', 'movies', 'television', 'music'])

In [3]:
articles.head()

,article_id,source_id,source_name,author,title,description,url,url_to_image,published_at,content,category,category_specified
0,26016,None,NBCSports.com,NBC Sports,LeBron reiterates he wants to play with son Br...,"""I need to be on the floor with my boy, I got ...",https://nba.nbcsports.com/2023/01/07/lebron-re...,https://media.zenfs.com/en/nbcsports.com/d2bf4...,2023-01-07,2021 Hoophall West - Perry v Sierra Canyon\r\n...,sports,nba
1,26017,None,CBS Sports,CBS Sports Staff,"Celtics vs. Spurs prediction, odds, line, spre...",SportsLine's computer model simulated San Anto...,https://www.cbssports.com/nba/news/celtics-vs-...,https://sportshub.cbsistatic.com/i/r/2022/06/1...,2023-01-07,The Boston Celtics (27-12) and the San Antonio...,sports,nba
2,26018,None,CBS Sports,Scout Staff,"Mavericks vs. Pelicans odds, line: 2023 NBA pi...",SportsLine's model just revealed its NBA picks...,https://www.cbssports.com/nba/news/mavericks-v...,https://sportshub.cbsistatic.com/i/r/2022/12/2...,2023-01-07,The New Orleans Pelicans and the Dallas Maveri...,sports,nba
3,26019,None,CBS Sports,Scout Staff,"Lakers vs. Kings odds, line, spread: 2023 NBA ...",SportsLine's model just revealed its NBA picks...,https://www.cbssports.com/nba/news/lakers-vs-k...,https://sportshub.cbsistatic.com/i/r/2023/01/0...,2023-01-07,The Los Angeles Lakers will try to extend thei...,sports,nba
4,26020,None,CBS Sports,Jack Maloney,LeBron James reconfirms plans to play until hi...,"Bronny James, currently a high school senior, ...",https://www.cbssports.com/nba/news/lebron-jame...,https://sportshub.cbsistatic.com/i/r/2022/09/0...,2023-01-07,The Los Angeles Lakers are on a bit of an upsw...,sports,nba


In [4]:
likedPosts['liked'] = likedPosts['liked'].apply(lambda x: 2 if x else 0)
dislikedPosts['disliked'] = dislikedPosts['disliked'].apply(lambda x: -1 if x else 0)
save_for_laterPosts['save_for_later'] = save_for_laterPosts['save_for_later'].apply(lambda x: 1 if x else 0)

In [5]:
preferenceProfileEncoded = preferenceProfile.copy()

for col in preferenceProfile.columns:
    if col == 'user_id':
        continue
    preferenceProfileEncoded[col] = preferenceProfileEncoded[col].astype(int)


In [6]:
preferenceProfileEncoded.head()

,user_id,business,politics,right_wing,left_wing,center,sports,nfl,nba,nhl,mma,entertainment,movies,television,music
0,4,1,1,1,0,1,1,1,1,0,1,1,1,0,1


In [7]:
likedPosts.head()

,liked_posts_id,user_id,article_id,liked
0,17,4,27368,0
1,18,4,27370,0
2,22,4,27372,0


In [8]:
dislikedPosts.head()

,dislike_id,user_id,article_id,disliked


In [9]:
save_for_laterPosts.head()

,save_for_later_id,user_id,article_id,save_for_later


In [10]:
preferenceProfile.head()

,user_id,business,politics,right_wing,left_wing,center,sports,nfl,nba,nhl,mma,entertainment,movies,television,music
0,4,True,True,True,False,True,True,True,True,False,True,True,True,False,True


In [11]:
preferenceProfileEncoded.head()

,user_id,business,politics,right_wing,left_wing,center,sports,nfl,nba,nhl,mma,entertainment,movies,television,music
0,4,1,1,1,0,1,1,1,1,0,1,1,1,0,1


In [12]:
likesdisliked = pd.merge(dislikedPosts, likedPosts, on=['user_id', 'article_id'], how='outer')

In [13]:
likesDislikesSaveForLaters = pd.merge(likesdisliked, save_for_laterPosts, on=['user_id', 'article_id'], how='outer')

In [14]:
likesDislikesSaveForLaters.fillna(0, inplace=True)

In [15]:
ratings = likesDislikesSaveForLaters.groupby('article_id').sum()

In [16]:
ratings.head(20)

,dislike_id,disliked,liked_posts_id,user_id,liked,save_for_later_id,save_for_later
article_id,,,,,,,
27368,0,0,17,4,0,0,0
27370,0,0,18,4,0,0,0
27372,0,0,22,4,0,0,0


In [17]:
articlesRatings = pd.merge(likesDislikesSaveForLaters, ratings, on='article_id', how='left')

In [18]:
articlesRatings.head(40)

,dislike_id_x,disliked_x,liked_posts_id_x,user_id_x,article_id,liked_x,save_for_later_id_x,save_for_later_x,dislike_id_y,disliked_y,liked_posts_id_y,user_id_y,liked_y,save_for_later_id_y,save_for_later_y
0,0,0,17,4,27368,0,0,0,0,0,17,4,0,0,0
1,0,0,18,4,27370,0,0,0,0,0,18,4,0,0,0
2,0,0,22,4,27372,0,0,0,0,0,22,4,0,0,0


In [19]:
articlesRatings['rating'] = articlesRatings[['disliked_x', 'liked_x', 'save_for_later_x', 'disliked_y', 'liked_y', 'save_for_later_y']].sum(axis=1)

In [20]:
articlesRatings.head(1000)

,dislike_id_x,disliked_x,liked_posts_id_x,user_id_x,article_id,liked_x,save_for_later_id_x,save_for_later_x,dislike_id_y,disliked_y,liked_posts_id_y,user_id_y,liked_y,save_for_later_id_y,save_for_later_y,rating
0,0,0,17,4,27368,0,0,0,0,0,17,4,0,0,0,0
1,0,0,18,4,27370,0,0,0,0,0,18,4,0,0,0,0
2,0,0,22,4,27372,0,0,0,0,0,22,4,0,0,0,0


In [21]:
articlesWithRatings = pd.merge(articlesRatings, articles, on = 'article_id', how='left')
articlesWithRatings.rename(columns={'user_id_x': 'user_id'}, inplace=True)

In [22]:
articlesWithPreferencesAndRatings = pd.merge(articlesWithRatings, preferenceProfileEncoded, on='user_id', how='left')

In [23]:
articlesWithPreferencesAndRatings.head()

,dislike_id_x,disliked_x,liked_posts_id_x,user_id,article_id,liked_x,save_for_later_id_x,save_for_later_x,dislike_id_y,disliked_y,liked_posts_id_y,user_id_y,liked_y,save_for_later_id_y,save_for_later_y,rating,source_id,source_name,author,title,description,url,url_to_image,published_at,content,category,category_specified,business,politics,right_wing,left_wing,center,sports,nfl,nba,nhl,mma,entertainment,movies,television,music
0,0,0,17,4,27368,0,0,0,0,0,17,4,0,0,0,0,None,Investing.com,Cointelegraph,Cathie Wood’s ARK enters 2023 with $5.7M Coinb...,Cathie Wood’s ARK enters 2023 with $5.7M Coinb...,https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/moved_L...,2023-01-07,We encourage you to use comments to engage wit...,business,None,1,1,1,0,1,1,1,1,0,1,1,1,0,1
1,0,0,18,4,27370,0,0,0,0,0,18,4,0,0,0,0,None,Investing.com,Reuters,Google challenges Android antitrust ruling in ...,Google challenges Android antitrust ruling in ...,https://www.investing.com/news/stock-market-ne...,https://i-invdn-com.investing.com/news/LYNXNPE...,2023-01-07,By Aditya Kalra and Arpan Chaturvedi\r\nNEW DE...,business,None,1,1,1,0,1,1,1,1,0,1,1,1,0,1
2,0,0,22,4,27372,0,0,0,0,0,22,4,0,0,0,0,None,Investing.com,CoinEdition,Perfect Buying Opportunity For Investors Inter...,Perfect Buying Opportunity For Investors Inter...,https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/Cryptoc...,2023-01-07,We encourage you to use comments to engage wit...,business,None,1,1,1,0,1,1,1,1,0,1,1,1,0,1


In [24]:
for index, row in articlesWithPreferencesAndRatings.iterrows():
    user_id = row['user_id']
    category = row['category']
    category_specified = row['category_specified']
    if category in articlesWithPreferencesAndRatings.columns:
        if preferenceProfile.loc[preferenceProfile['user_id'] == user_id, category].values[0] == 1:
            articlesWithPreferencesAndRatings.loc[index, 'rating'] += 1
    if category_specified in articlesWithPreferencesAndRatings.columns:
        if preferenceProfile.loc[preferenceProfile['user_id'] == user_id, category_specified].values[0] == 1:
            articlesWithPreferencesAndRatings.loc[index, 'rating'] += 0.5


In [25]:
articlesWithPreferencesAndRatings.head()

,dislike_id_x,disliked_x,liked_posts_id_x,user_id,article_id,liked_x,save_for_later_id_x,save_for_later_x,dislike_id_y,disliked_y,liked_posts_id_y,user_id_y,liked_y,save_for_later_id_y,save_for_later_y,rating,source_id,source_name,author,title,description,url,url_to_image,published_at,content,category,category_specified,business,politics,right_wing,left_wing,center,sports,nfl,nba,nhl,mma,entertainment,movies,television,music
0,0,0,17,4,27368,0,0,0,0,0,17,4,0,0,0,1,None,Investing.com,Cointelegraph,Cathie Wood’s ARK enters 2023 with $5.7M Coinb...,Cathie Wood’s ARK enters 2023 with $5.7M Coinb...,https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/moved_L...,2023-01-07,We encourage you to use comments to engage wit...,business,None,1,1,1,0,1,1,1,1,0,1,1,1,0,1
1,0,0,18,4,27370,0,0,0,0,0,18,4,0,0,0,1,None,Investing.com,Reuters,Google challenges Android antitrust ruling in ...,Google challenges Android antitrust ruling in ...,https://www.investing.com/news/stock-market-ne...,https://i-invdn-com.investing.com/news/LYNXNPE...,2023-01-07,By Aditya Kalra and Arpan Chaturvedi\r\nNEW DE...,business,None,1,1,1,0,1,1,1,1,0,1,1,1,0,1
2,0,0,22,4,27372,0,0,0,0,0,22,4,0,0,0,1,None,Investing.com,CoinEdition,Perfect Buying Opportunity For Investors Inter...,Perfect Buying Opportunity For Investors Inter...,https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/Cryptoc...,2023-01-07,We encourage you to use comments to engage wit...,business,None,1,1,1,0,1,1,1,1,0,1,1,1,0,1


In [26]:
articlesWithPreferencesAndRatings.drop(columns = ['dislike_id_x','disliked_x','liked_posts_id_x', 'liked_x', 'save_for_later_id_x','save_for_later_x', 'dislike_id_y', 'user_id_y','disliked_y','liked_posts_id_y','liked_y',
                                                 'save_for_later_id_y','save_for_later_y','source_id','source_name','author','title','description','url', 'url_to_image','published_at', 'content', 
                                                 'category', 'category_specified','business','politics','left_wing','right_wing','center','sports','nfl','nba', 'nhl', 'mma', 'entertainment','movies','television',
                                                 'music'], inplace = True)

In [27]:
articlesWithPreferencesAndRatings.rename(columns={'user_id': 'userId', 'article_id': 'itemId', 'rating': 'rating'}, inplace = True)

In [28]:
s = re.compile("\'s")  # matches `'s` from text  
rn = re.compile("\\r\\n") #matches `\r` and `\n`
punc = re.compile(r"[^\w\s]") # matches all non 0-9 A-z whitespace 
minWords = 4
maxWords = 200

In [29]:
def cleanText(text):

    text = text.lower()  # lowercase text
    # replace the matched string with ' '
    text = re.sub(s, ' ', text)
    text = re.sub(rn, ' ', text)
    text = re.sub(punc, ' ', text)
    return text

In [30]:
def tokenizer(sentence, min_words=minWords, max_words=maxWords, stopwords=stopWords, lemmatize=True):

    if lemmatize:
        stemmer = WordNetLemmatizer()
        tokens = [stemmer.lemmatize(w) for w in word_tokenize(sentence)]
    else:
        tokens = [w for w in word_tokenize(sentence)]
    token = [w for w in tokens if (len(w) > min_words and len(w) < max_words
                                                        and w not in stopwords)]
    return tokens  

In [31]:
def cleanSentences(df):

    print('Cleaning sentences...')
    df['clean_sentence'] = df['description'].apply(cleanText)
    df['tok_lem_sentence'] = df['clean_sentence'].apply(
        lambda x: tokenizer(x, min_words=minWords, max_words=maxWords, stopwords=stopWords))
    return df

In [32]:
articles['clean_title'] = articles['title'].fillna('').apply(cleanText)
articles['clean_description'] = articles['description'].fillna('').apply(cleanText)
articles['clean_content'] = articles['content'].fillna('').apply(cleanText)

articles['tok_lem_title'] = articles['clean_title'].apply(
    lambda x: tokenizer(x, min_words=minWords, max_words=maxWords, stopwords=stopWords))
articles['tok_lem_description'] = articles['clean_description'].apply(
    lambda x: tokenizer(x, min_words=minWords, max_words=maxWords, stopwords=stopWords))
articles['tok_lem_content'] = articles['clean_content'].apply(
    lambda x: tokenizer(x, min_words=minWords, max_words=maxWords, stopwords=stopWords))


In [33]:
articles.head()

,article_id,source_id,source_name,author,title,description,url,url_to_image,published_at,content,category,category_specified,clean_title,clean_description,clean_content,tok_lem_title,tok_lem_description,tok_lem_content
0,26016,None,NBCSports.com,NBC Sports,LeBron reiterates he wants to play with son Br...,"""I need to be on the floor with my boy, I got ...",https://nba.nbcsports.com/2023/01/07/lebron-re...,https://media.zenfs.com/en/nbcsports.com/d2bf4...,2023-01-07,2021 Hoophall West - Perry v Sierra Canyon\r\n...,sports,nba,lebron reiterates he wants to play with son br...,i need to be on the floor with my boy i got ...,2021 hoophall west perry v sierra canyon hig...,"[lebron, reiterates, he, want, to, play, with,...","[i, need, to, be, on, the, floor, with, my, bo...","[2021, hoophall, west, perry, v, sierra, canyo..."
1,26017,None,CBS Sports,CBS Sports Staff,"Celtics vs. Spurs prediction, odds, line, spre...",SportsLine's computer model simulated San Anto...,https://www.cbssports.com/nba/news/celtics-vs-...,https://sportshub.cbsistatic.com/i/r/2022/06/1...,2023-01-07,The Boston Celtics (27-12) and the San Antonio...,sports,nba,celtics vs spurs prediction odds line spre...,sportsline computer model simulated san anton...,the boston celtics 27 12 and the san antonio...,"[celtic, v, spur, prediction, odds, line, spre...","[sportsline, computer, model, simulated, san, ...","[the, boston, celtic, 27, 12, and, the, san, a..."
2,26018,None,CBS Sports,Scout Staff,"Mavericks vs. Pelicans odds, line: 2023 NBA pi...",SportsLine's model just revealed its NBA picks...,https://www.cbssports.com/nba/news/mavericks-v...,https://sportshub.cbsistatic.com/i/r/2022/12/2...,2023-01-07,The New Orleans Pelicans and the Dallas Maveri...,sports,nba,mavericks vs pelicans odds line 2023 nba pi...,sportsline model just revealed its nba picks ...,the new orleans pelicans and the dallas maveri...,"[maverick, v, pelican, odds, line, 2023, nba, ...","[sportsline, model, just, revealed, it, nba, p...","[the, new, orleans, pelican, and, the, dallas,..."
3,26019,None,CBS Sports,Scout Staff,"Lakers vs. Kings odds, line, spread: 2023 NBA ...",SportsLine's model just revealed its NBA picks...,https://www.cbssports.com/nba/news/lakers-vs-k...,https://sportshub.cbsistatic.com/i/r/2023/01/0...,2023-01-07,The Los Angeles Lakers will try to extend thei...,sports,nba,lakers vs kings odds line spread 2023 nba ...,sportsline model just revealed its nba picks ...,the los angeles lakers will try to extend thei...,"[lakers, v, king, odds, line, spread, 2023, nb...","[sportsline, model, just, revealed, it, nba, p...","[the, los, angeles, lakers, will, try, to, ext..."
4,26020,None,CBS Sports,Jack Maloney,LeBron James reconfirms plans to play until hi...,"Bronny James, currently a high school senior, ...",https://www.cbssports.com/nba/news/lebron-jame...,https://sportshub.cbsistatic.com/i/r/2022/09/0...,2023-01-07,The Los Angeles Lakers are on a bit of an upsw...,sports,nba,lebron james reconfirms plans to play until hi...,bronny james currently a high school senior ...,the los angeles lakers are on a bit of an upsw...,"[lebron, james, reconfirms, plan, to, play, un...","[bronny, james, currently, a, high, school, se...","[the, los, angeles, lakers, are, on, a, bit, o..."


In [34]:
articles = articles.drop(['title', 'description', 'content','url','url_to_image','source_id','clean_title','clean_description','clean_content'], axis=1)

In [35]:
articles.head()

,article_id,source_name,author,published_at,category,category_specified,tok_lem_title,tok_lem_description,tok_lem_content
0,26016,NBCSports.com,NBC Sports,2023-01-07,sports,nba,"[lebron, reiterates, he, want, to, play, with,...","[i, need, to, be, on, the, floor, with, my, bo...","[2021, hoophall, west, perry, v, sierra, canyo..."
1,26017,CBS Sports,CBS Sports Staff,2023-01-07,sports,nba,"[celtic, v, spur, prediction, odds, line, spre...","[sportsline, computer, model, simulated, san, ...","[the, boston, celtic, 27, 12, and, the, san, a..."
2,26018,CBS Sports,Scout Staff,2023-01-07,sports,nba,"[maverick, v, pelican, odds, line, 2023, nba, ...","[sportsline, model, just, revealed, it, nba, p...","[the, new, orleans, pelican, and, the, dallas,..."
3,26019,CBS Sports,Scout Staff,2023-01-07,sports,nba,"[lakers, v, king, odds, line, spread, 2023, nb...","[sportsline, model, just, revealed, it, nba, p...","[the, los, angeles, lakers, will, try, to, ext..."
4,26020,CBS Sports,Jack Maloney,2023-01-07,sports,nba,"[lebron, james, reconfirms, plan, to, play, un...","[bronny, james, currently, a, high, school, se...","[the, los, angeles, lakers, are, on, a, bit, o..."


In [36]:
stemmer = PorterStemmer()

articles['tok_lem_title'] = articles['tok_lem_title'].apply(lambda x: [stemmer.stem(word) for word in x])
articles['tok_lem_description'] = articles['tok_lem_description'].apply(lambda x: [stemmer.stem(word) for word in x])
articles['tok_lem_content'] = articles['tok_lem_content'].apply(lambda x: [stemmer.stem(word) for word in x])


In [37]:
articles.head()

,article_id,source_name,author,published_at,category,category_specified,tok_lem_title,tok_lem_description,tok_lem_content
0,26016,NBCSports.com,NBC Sports,2023-01-07,sports,nba,"[lebron, reiter, he, want, to, play, with, son...","[i, need, to, be, on, the, floor, with, my, bo...","[2021, hoophal, west, perri, v, sierra, canyon..."
1,26017,CBS Sports,CBS Sports Staff,2023-01-07,sports,nba,"[celtic, v, spur, predict, odd, line, spread, ...","[sportslin, comput, model, simul, san, antonio...","[the, boston, celtic, 27, 12, and, the, san, a..."
2,26018,CBS Sports,Scout Staff,2023-01-07,sports,nba,"[maverick, v, pelican, odd, line, 2023, nba, p...","[sportslin, model, just, reveal, it, nba, pick...","[the, new, orlean, pelican, and, the, dalla, m..."
3,26019,CBS Sports,Scout Staff,2023-01-07,sports,nba,"[laker, v, king, odd, line, spread, 2023, nba,...","[sportslin, model, just, reveal, it, nba, pick...","[the, lo, angel, laker, will, tri, to, extend,..."
4,26020,CBS Sports,Jack Maloney,2023-01-07,sports,nba,"[lebron, jame, reconfirm, plan, to, play, unti...","[bronni, jame, current, a, high, school, senio...","[the, lo, angel, laker, are, on, a, bit, of, a..."


In [38]:

articlesWithPreferencesAndRatings['tok_lem_all'] = articles['tok_lem_title'] + articles['tok_lem_description'] + articles['tok_lem_content']


articlesWithPreferencesAndRatings['tok_lem_all'] = articlesWithPreferencesAndRatings['tok_lem_all'].apply(lambda x: ' '.join(x))

tf = TfidfVectorizer(analyzer = 'word', ngram_range = (1,2), min_df = 0, stop_words = 'english')

tfidfMatrix = tf.fit_transform(articlesWithPreferencesAndRatings['tok_lem_all'])

cosine_sim = cosine_similarity(tfidfMatrix, tfidfMatrix)